# Puntuación crediticia

Detalles del proyecto

Objetivo: Deberán desarrollar un modelo de crédito tradicional en Excel o Python. 

Este documento debe estar estructurado de manera que se le alimente una lista de n solicitantes en el mismo formato y evaluemos posteriormente el desempeño del modelo, el desempeño siendo la similitud entre calificación otorgada y la clasificada en el documento.

El trabajo escrito debe contar con Introducción, marco teórico (explicando su Tablero de Puntaje de crédito), y conclusión. Se debe en este documento escrito presentar los resultados obtenidos, cuántos errores se obtuvieron, etc. Finalmente, en el documento escrito, una reflexión por parte del equipo sobre los mayores retos al realizar este trabajo. 

Requisitos:

Documento escrito explicando su modelo. \
Debe explicar los supuestos de manera concreta. - 6pts\
Como anexo debe venir su código o documento de excel. - 6 pts

### Librerias y Data

In [1]:
import numpy as np
import pandas as pd 

In [2]:
Data = pd.read_csv("train.csv")#,dtype=str

/Users/Beto/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Limpieza de Datos

In [3]:
#Age
Data['Age'] = Data['Age'].str.replace(',', '').apply(lambda x: x.replace('_', '') if isinstance(x, str) else x)
Data['Age'] = pd.to_numeric(Data['Age'], errors='coerce')

Data['Age'] = Data['Age'].apply(lambda x: x if (isinstance(x, (int, float)) and 0 <= x <= 100) else None)
Data['Age'] = Data['Age'].fillna('NaN')

#Occupation
Formato_OC = '_______'
Data['Occupation'] = Data['Occupation'].apply(lambda x: None if str(x) == Formato_OC else x) 
Data['Occupation'] = Data['Occupation'].fillna('NaN')

#Annual_Income
Data['Annual_Income'] = Data['Annual_Income'].apply(lambda x: str(x).replace('_', ' ') if isinstance(x, str) else x)

#Num_Credit_Card
Data['Num_Credit_Card'] = Data['Num_Credit_Card'].apply(lambda x: x if (isinstance(x, (int, float)) and 0 <= x <= 10) else None)
Data['Num_Credit_Card'] = Data['Num_Credit_Card'].fillna('NaN')

#Delay_from_due_date
Data['Delay_from_due_date'] = pd.to_numeric(Data['Delay_from_due_date'], errors='coerce')
Data['Delay_from_due_date'] = Data['Delay_from_due_date'].apply(lambda x: x if (isinstance(x, (int, float)) and 0 <= x <= 10_000) else None)
Data['Delay_from_due_date'] = Data['Delay_from_due_date'].fillna(0)

#Credit_Mix
formato_CMix = '_'
Data['Credit_Mix'] = Data['Credit_Mix'].apply(lambda x: None if str(x) == formato_CMix else x)

#Outstanding_Debt
Data['Outstanding_Debt'] = Data['Outstanding_Debt'].str.replace(',', '').str.replace('_', '')

#Payment_Behaviour
formato_PB=['High_spent_Small_value_payments','Low_spent_Large_value_payments','Low_spent_Medium_value_payments',
            'Low_spent_Small_value_payments','High_spent_Medium_value_payments','High_spent_Large_value_payments']
Data['Payment_Behaviour']=Data['Payment_Behaviour'].apply(lambda x: x if x in formato_PB else np.nan)

In [4]:
Data.to_excel('DataFrameModelos_Puntuacion.xlsx', index=False)
data_frame = pd.read_excel('DataFrameModelos_Puntuacion.xlsx')

### Modelo crediticio Valoracion

Crear una nueva columna llamada puntaje donde, a base de puntos, se medirá el score.

In [5]:
mc = data_frame.drop(['Name', 
                      'Customer_ID',
                      'Month',
                      'SSN',
                      'Monthly_Inhand_Salary',
                      'Num_Bank_Accounts',
                      'Interest_Rate',
                      'Num_of_Loan',
                      'Type_of_Loan',
                      'Num_of_Delayed_Payment',
                      'Changed_Credit_Limit',
                      'Num_Credit_Inquiries',
                      'Credit_Utilization_Ratio',
                      'Credit_History_Age',
                      'Total_EMI_per_month',
                      'Amount_invested_monthly',
                      'Monthly_Balance'], axis=1)

mc['Score'] = 0

In [39]:
#Pruebas para ver el comportamiento de cada columna

#Data['Annual_Income'] = pd.to_numeric(Data['Annual_Income'], errors='coerce')
#x=Data[Data['Credit_Score']>='Poor']
#y=x['Payment_Behaviour'].value_counts()

In [6]:
    #Filtro Occupation
def asignar_puntaje_occupation(trabajo):
        if trabajo == 'NaN':
            return 0 
        else:
            return 300
        
mc['Score'] += mc['Occupation'].apply(asignar_puntaje_occupation)

#Filtro Annual_income 
mc['Annual_Income'] = pd.to_numeric(mc['Annual_Income'], errors='coerce')
mean_income = mc['Annual_Income'].mean()

def asignar_puntaje_income(income):
        if income > 99_999:
            return 500
        elif 99_999<=income <=9_999:
            return 300
        else:
            return 100
        
mc['Score'] += mc['Annual_Income'].apply(asignar_puntaje_income)

    #Filtro Delay_from_due_date
mc['Delay_from_due_date'] = pd.to_numeric(mc['Delay_from_due_date'], errors='coerce')

def asignar_puntaje_Delay_from_due_date(Delay_from_due_date):
        if Delay_from_due_date == [8,14,10]:
            return 200
        elif Delay_from_due_date == [15,13,9,12]:
            return 100
        elif Delay_from_due_date == [27,19,9]:
            return 0
        else:
            return 80
        
mc['Score'] += mc['Delay_from_due_date'].apply(asignar_puntaje_Delay_from_due_date)

    #Filtro Credit_Mix 
def asignar_puntaje_Credit_mix(Credit_mix):
        if Credit_mix == 'Bad':
            return 50 
        elif Credit_mix =='Standard':
            return 80
        elif Credit_mix =='Good':
            return 300
        else:
            return 0
        
mc['Score'] += mc['Credit_Mix'].apply(asignar_puntaje_Credit_mix)    
    
    #Filtro Outstanding_Debt
mc['Outstanding_Debt'] = pd.to_numeric(mc['Outstanding_Debt'], errors='coerce')
Outstanding_Debt_mean = mc['Outstanding_Debt'].mean()

def asignar_puntaje_Outstanding_Debt(Outstanding_Debt):
        if Outstanding_Debt < 1166.155000:
            return 300
        elif 1166.155000 <= Outstanding_Debt < 1426.220376 :
            return 150
        elif 1426.220376 <= Outstanding_Debt < 1945.962500:
            return 100
        else: 
            return 50
        
mc['Score'] += mc['Outstanding_Debt'].apply(asignar_puntaje_Outstanding_Debt)

In [8]:
mc['Score'].isna().sum()

0

In [7]:
mc['Score'].describe()

count    100000.000000
mean        840.608700
std         255.706141
min         530.000000
25%         610.000000
50%         780.000000
75%        1080.000000
max        1480.000000
Name: Score, dtype: float64

In [37]:
mc['Final'] = 0
mc['Score'] = pd.to_numeric(mc['Score'], errors='coerce')

def asignar_puntaje_score(score):
        if score < 650: #score < 635
            return "Poor"
        elif 650 <= score < 1075: #635 <= score < 1075
            return "Standard"
        elif 1075 <= score:  #1075 <= score
            return "Good"

mc['Final'] = mc['Score'].apply(asignar_puntaje_score) #57,58
mc['Final'].value_counts()

Standard    41639
Poor        31972
Good        26389
Name: Final, dtype: int64

In [45]:
mc.to_excel('Modelo_Crediticio_MMB.xlsx', index=False)

## Cantidad total de puntos 670

In [38]:
accuracy=(data_frame['Credit_Score']==mc['Final']).sum()
total_filas = len(data_frame)
precision = accuracy / total_filas *100
print("Precisión:", precision, '%')

Precisión: 58.458 %
